In [1]:
import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

c:\Users\Subhash\anaconda3\envs\myenv3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:

# Load the GROQ and OpenAI API keys
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

st.title("Gemma Model Document Q&A")

#llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")


2024-07-20 17:40:29.456 
  command:

    streamlit run c:\Users\Subhash\anaconda3\envs\myenv3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [3]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on the provided context only.
    Please provide the most accurate response based on the question
    <context>
    {context}
    <context>
    Questions:{input}
    """
)

In [15]:
pwd

'c:\\Users\\Subhash\\OneDrive\\Desktop\\GENAI\\GEMMA'

In [16]:
st.session_state.embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        
        # Data Ingestion
st.session_state.loader = PyPDFDirectoryLoader(".\pdf_downloads")
st.session_state.docs = st.session_state.loader.load()
        
        # Debugging: Check the number of documents loaded
st.write(f"Number of documents loaded: {len(st.session_state.docs)}")

<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Subhash\AppData\Local\Temp\ipykernel_10220\578626586.py:4: SyntaxWarning: invalid escape sequence '\p'
  st.session_state.loader = PyPDFDirectoryLoader(".\pdf_downloads")


In [17]:
st.session_state.docs

[Document(metadata={'source': 'pdf_downloads\\ICICI%20Future%20Perfect%20-%20Brochure.pdf', 'page': 0}, page_content='Get life cover to protect your family ’s future\nA safe journey to a rising future'),
 Document(metadata={'source': 'pdf_downloads\\ICICI%20Future%20Perfect%20-%20Brochure.pdf', 'page': 1}, page_content='Key benefits\nICICI P ru Future P erfect provides you:\n•   Savings with the comfort of guarantees  - \n     At maturity of the policy , you receive:\n»   Guaranteed Maturity Benefit (GMB)\n»   Accrued Guaranteed Additions (GAs) \n»   Vested reversionary bonuses ,  if any\n»   Terminal bonus , if any\n•  Protection  - Get life cover for the entire policy term\n•  Tax benefits – Tax benefits apply to premiums paid T&C3\n    and benefits received as per the prevailing tax laws •  Flexibility  - Choose premium payment term, premium \n     payment frequency , Sum Assured on death and policy \n     term as per your needIn our day to day lives, we strive towards achieving our

In [4]:

def vector_embedding():
    if "vectors" not in st.session_state:
        st.session_state.embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        
        # Data Ingestion
        st.session_state.loader = PyPDFDirectoryLoader("./pdf_dwonloads")
        st.session_state.docs = st.session_state.loader.load()
        
        # Debugging: Check the number of documents loaded
        st.write(f"Number of documents loaded: {len(st.session_state.docs)}")
        
        if len(st.session_state.docs) == 0:
            st.error("No documents found. Please check the directory and try again.")
            return

        # Chunk Creation
        st.session_state.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        st.session_state.final_documents = st.session_state.text_splitter.split_documents(st.session_state.docs[:20])
        
        # Debugging: Check the number of chunks created
        st.write(f"Number of chunks created: {len(st.session_state.final_documents)}")
        
        if len(st.session_state.final_documents) == 0:
            st.error("No document chunks created. Please check the splitting logic.")
            return
        
        # Vector Embedding
        try:
            st.session_state.vectors = FAISS.from_documents(st.session_state.final_documents, st.session_state.embeddings)
            st.write("Vector Store DB is ready")
        except IndexError as e:
            st.error(f"Error creating vector store: {e}")
            st.write("Check if the embeddings are being generated correctly.")

prompt1 = st.text_input("Enter Your Question From Documents")

if st.button("Documents Embedding"):
    vector_embedding()

import time

if prompt1:
    document_chain = create_stuff_documents_chain(llm, prompt)
    retriever = st.session_state.vectors.as_retriever()
    retrieval_chain = create_retrieval_chain(retriever, document_chain)
    
    start = time.process_time()
    response = retrieval_chain.invoke({'input': prompt1})
    st.write("Response time: ", time.process_time() - start)
    st.write(response['answer'])

    # With a streamlit expander
    with st.expander("Document Similarity Search"):
        # Find the relevant chunks
        for i, doc in enumerate(response["context"]):
            st.write(doc.page_content)
            st.write("--------------------------------")

2024-07-20 17:41:05.617 Session state does not function when running a script without `streamlit run`
